# MLT Alpha from Flux Balance: Interactive Exploration

This notebook provides an interactive interface to explore the mixing-length parameter α by solving the flux balance equation:

$$F_c(\alpha) = F_{need} = F_{tot} - F_{rad}$$

Where the convective flux is:
$$F_c(\alpha) = \rho c_p T S(\alpha) (\nabla - \nabla_{ad})^{3/2}$$

with:
$$S(\alpha) = \sqrt{\frac{g\delta}{8H_p}} \cdot (\alpha H_p)^2$$

## Sections:
1. Single layer test (adjustable parameters)
2. Sensitivity to F_need
3. T/P parameter space scan
4. Visualization
5. Export results


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mlt_flux_balance import calculate_alpha_from_flux, compute_gradient, compute_adiabatic_gradient
from explore_tp_parameter_space import explore_parameter_space, generate_layer_from_profile, plot_results

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline


## 1. Single Layer Test

Test the flux balance calculation with adjustable parameters.


In [ ]:
# === ADJUST THESE PARAMETERS ===

# Layer properties
layer_data = {
    'T_top': 1000.0,     # K
    'T_mid': 1500.0,     # K
    'T_bot': 2250.0,     # K
    'P_top': 1e4,        # Pa
    'P_mid': 5e4,        # Pa
    'P_bot': 1e5,        # Pa
}

# Flux data
flux_data = {
    'F_tot': 1e7,        # W/m² - total flux
    'F_rad': 5e6,        # W/m² - radiative component
}

# Physical parameters
physical_params = {
    'g': 10.0,           # m/s² - gravitational acceleration
    'delta': 1.0,        # dimensionless - thermodynamic parameter (1 for ideal gas)
    'R_universal': 8.314, # J/(mol·K) - universal gas constant
    'mu': 0.0022,        # kg/mol - mean molecular weight
    'c_p': 14000.0,      # J/(kg·K) - specific heat capacity
    'rho': 0.005,        # kg/m³ - density
}

# ================================

# Calculate alpha
result = calculate_alpha_from_flux(layer_data, flux_data, physical_params, verbose=True)

print("\n" + "="*60)
print("SOLUTION SUMMARY")
print("="*60)
if result['alpha'] is not None:
    print(f"✓ Mixing-length parameter: α = {result['alpha']:.4f}")
    print(f"✓ Mixing length: l = {result['alpha'] * result['H_p']:.2f} m")
    print(f"✓ Pressure scale height: H_p = {result['H_p']:.2f} m")
    print(f"✓ l/H_p ratio: {result['alpha']:.4f}")
    print(f"✓ Convective flux: F_c = {result['F_c']:.2e} W/m²")
    print(f"✓ Required flux: F_need = {result['F_need']:.2e} W/m²")
else:
    print(f"✗ {result['convergence_info']}")
print("="*60)


## 2. Sensitivity to Required Flux

Explore how α varies with F_need (= F_tot - F_rad).


In [ ]:
# Keep layer fixed, vary F_need
layer_fixed = layer_data.copy()
params_fixed = physical_params.copy()

# Range of F_need values
F_tot_fixed = 1e7
F_rad_range = np.linspace(0.1e7, 0.9e7, 30)  # Vary F_rad to change F_need
F_need_range = F_tot_fixed - F_rad_range

alpha_vs_flux = []

for F_rad in F_rad_range:
    flux_test = {'F_tot': F_tot_fixed, 'F_rad': F_rad}
    result = calculate_alpha_from_flux(layer_fixed, flux_test, params_fixed, verbose=False)
    alpha_vs_flux.append(result['alpha'] if result['alpha'] is not None else np.nan)

alpha_vs_flux = np.array(alpha_vs_flux)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Alpha vs F_need
axes[0].plot(F_need_range/1e6, alpha_vs_flux, 'o-', linewidth=2, markersize=6, color='steelblue')
axes[0].set_xlabel(r'$F_{need} = F_{tot} - F_{rad}$ [MW/m²]', fontsize=12)
axes[0].set_ylabel(r'$\alpha$', fontsize=12)
axes[0].set_title(r'Mixing-Length Parameter vs Required Flux', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].set_yscale('log')
axes[0].set_xscale('log')

# Mixing length vs F_need
l_vs_flux = alpha_vs_flux * result['H_p']
axes[1].plot(F_need_range/1e6, l_vs_flux/1000, 'o-', linewidth=2, markersize=6, color='darkorange')
axes[1].set_xlabel(r'$F_{need}$ [MW/m²]', fontsize=12)
axes[1].set_ylabel(r'Mixing Length $l$ [km]', fontsize=12)
axes[1].set_title(r'Mixing Length vs Required Flux', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].set_yscale('log')
axes[1].set_xscale('log')

plt.tight_layout()
plt.show()

print(f"Explored F_need range: [{F_need_range.min():.2e}, {F_need_range.max():.2e}] W/m²")
print(f"Resulting α range: [{np.nanmin(alpha_vs_flux):.4f}, {np.nanmax(alpha_vs_flux):.4f}]")


In [ ]:
# Define parameter ranges for exploration
print("Setting up parameter space...")

# Temperature profiles
T_top_vals = np.linspace(1000, 1500, 15)
T_bot_vals = np.linspace(1500, 2500, 15)
T_range = [(T_top, T_bot) for T_top in T_top_vals for T_bot in T_bot_vals if T_bot > T_top + 200]

# Pressure range
P_top_range = np.logspace(3, 5, 12)  # 1 kPa to 100 kPa
P_bot_fixed = 1e5  # Fixed at 100 kPa

# Flux and physical parameters (use values from earlier)
flux_exploration = {'F_tot': 1e7, 'F_rad': 5e6}
params_exploration = physical_params.copy()

print(f"Temperature profiles: {len(T_range)}")
print(f"Pressure levels: {len(P_top_range)}")
print(f"Total calculations: {len(T_range) * len(P_top_range)}")
print("\nRunning exploration (this may take a minute)...")


In [ ]:
%%time
# Run parameter space exploration
results = explore_parameter_space(T_range, P_top_range, P_bot_fixed,
                                 flux_exploration, params_exploration, verbose=False)

print("\nExploration complete!")
print(f"Superadiabatic cases: {np.sum(results['superadiabatic_grid'])}")

alpha_valid = results['alpha_grid'][results['superadiabatic_grid'] & ~np.isnan(results['alpha_grid'])]
if len(alpha_valid) > 0:
    print(f"Valid α solutions: {len(alpha_valid)}")
    print(f"  Mean: {np.nanmean(alpha_valid):.4f}")
    print(f"  Median: {np.nanmedian(alpha_valid):.4f}")
    print(f"  Range: [{np.nanmin(alpha_valid):.4f}, {np.nanmax(alpha_valid):.4f}]")


## 4. Visualization

Create comprehensive plots of the parameter space results.


In [ ]:
# Extract data for plotting
T_top_vals_plot = np.array([t[0] for t in results['T_range']])
T_bot_vals_plot = np.array([t[1] for t in results['T_range']])
delta_T_vals = T_bot_vals_plot - T_top_vals_plot

# Create meshgrids
Delta_T_mesh, P_top_mesh = np.meshgrid(delta_T_vals, results['P_top_range'], indexing='ij')

# Main heatmap
fig, ax = plt.subplots(figsize=(10, 8))

alpha_masked = np.ma.masked_where(~results['superadiabatic_grid'], results['alpha_grid'])
im = ax.contourf(Delta_T_mesh, P_top_mesh/1e3, alpha_masked, levels=20, cmap='viridis')
cbar = plt.colorbar(im, ax=ax, label=r'$\alpha$')

# Add contour lines
contour_lines = ax.contour(Delta_T_mesh, P_top_mesh/1e3, alpha_masked,
                           levels=8, colors='white', alpha=0.4, linewidths=0.5)
ax.clabel(contour_lines, inline=True, fontsize=8, fmt='%.3f')

ax.set_xlabel(r'$\Delta T = T_{bot} - T_{top}$ [K]', fontsize=12)
ax.set_ylabel(r'$P_{top}$ [kPa]', fontsize=12)
ax.set_yscale('log')
ax.set_title(r'MLT Parameter $\alpha$ from Flux Balance' + '\n' +
            f'F_tot={flux_exploration["F_tot"]:.1e} W/m², F_rad={flux_exploration["F_rad"]:.1e} W/m²',
            fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Additional plots
fig, axes = plt.subplots(2, 2, figsize=(14, 11))

# Plot 1: ∇ vs ∇_ad
ax = axes[0, 0]
im1 = ax.contourf(Delta_T_mesh, P_top_mesh/1e3, results['nabla_grid'], levels=20, cmap='plasma')
cbar1 = plt.colorbar(im1, ax=ax, label=r'$\nabla$')
nabla_ad_mean = np.nanmean(results['nabla_ad_grid'])
CS = ax.contour(Delta_T_mesh, P_top_mesh/1e3, results['nabla_ad_grid'],
               levels=[nabla_ad_mean], colors='white', linewidths=2)
ax.clabel(CS, inline=True, fontsize=9, fmt=r'$\nabla_{ad}$=%.3f')
ax.set_xlabel(r'$\Delta T$ [K]', fontsize=11)
ax.set_ylabel(r'$P_{top}$ [kPa]', fontsize=11)
ax.set_yscale('log')
ax.set_title(r'Temperature Gradient $\nabla$', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Plot 2: Convective flux
ax = axes[0, 1]
F_c_masked = np.ma.masked_where(~results['superadiabatic_grid'], results['F_c_grid'])
im2 = ax.contourf(Delta_T_mesh, P_top_mesh/1e3, F_c_masked/1e6, levels=20, cmap='coolwarm')
cbar2 = plt.colorbar(im2, ax=ax, label=r'$F_c$ [MW/m²]')
ax.set_xlabel(r'$\Delta T$ [K]', fontsize=11)
ax.set_ylabel(r'$P_{top}$ [kPa]', fontsize=11)
ax.set_yscale('log')
ax.set_title(r'Convective Flux $F_c$', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Plot 3: Alpha distribution
ax = axes[1, 0]
if len(alpha_valid) > 0:
    ax.hist(alpha_valid, bins=25, edgecolor='black', alpha=0.7, color='steelblue')
    ax.axvline(np.nanmean(alpha_valid), color='red', linestyle='--', linewidth=2,
              label=f'Mean = {np.nanmean(alpha_valid):.4f}')
    ax.axvline(np.nanmedian(alpha_valid), color='orange', linestyle='--', linewidth=2,
              label=f'Median = {np.nanmedian(alpha_valid):.4f}')
    ax.set_xlabel(r'$\alpha$', fontsize=11)
    ax.set_ylabel('Frequency', fontsize=11)
    ax.set_title(r'Distribution of $\alpha$', fontsize=12, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3, axis='y')

# Plot 4: Alpha vs ΔT slices
ax = axes[1, 1]
n_curves = min(4, len(results['P_top_range']))
indices = np.linspace(0, len(results['P_top_range'])-1, n_curves, dtype=int)
for idx in indices:
    P_val = results['P_top_range'][idx]
    ax.plot(delta_T_vals, results['alpha_grid'][:, idx], 'o-', 
           label=f'P_top = {P_val/1e3:.1f} kPa', markersize=4)
ax.set_xlabel(r'$\Delta T$ [K]', fontsize=11)
ax.set_ylabel(r'$\alpha$', fontsize=11)
ax.set_title(r'$\alpha$ vs $\Delta T$ (at different P)', fontsize=12, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 5. Export Results

Save results to file for use in your radiative transfer code.


In [ ]:
# Save results to numpy archive
filename = 'mlt_alpha_results.npz'
np.savez(filename, **results)
print(f"✓ Results saved to: {filename}")

# Summary for RT code implementation
print("\n" + "="*60)
print("RECOMMENDATIONS FOR YOUR RT CODE")
print("="*60)

if len(alpha_valid) > 0:
    print(f"\nBased on parameter space exploration:")
    print(f"  Recommended α (mean): {np.nanmean(alpha_valid):.4f}")
    print(f"  Recommended α (median): {np.nanmedian(alpha_valid):.4f}")
    print(f"  Standard deviation: {np.nanstd(alpha_valid):.4f}")
    print(f"  Safe range: [{np.percentile(alpha_valid, 10):.4f}, {np.percentile(alpha_valid, 90):.4f}]")
    print(f"\nThis was calculated for:")
    print(f"  F_tot = {flux_exploration['F_tot']:.2e} W/m²")
    print(f"  F_rad = {flux_exploration['F_rad']:.2e} W/m²")
    print(f"  F_need = {flux_exploration['F_tot'] - flux_exploration['F_rad']:.2e} W/m²")
    print(f"\nPhysical parameters used:")
    print(f"  g = {params_exploration['g']} m/s²")
    print(f"  μ = {params_exploration['mu']} kg/mol")
    print(f"  c_p = {params_exploration['c_p']} J/(kg·K)")
    
    print(f"\nTo use in your RT code:")
    print(f"  1. For each layer, calculate F_need = F_tot - F_rad")
    print(f"  2. Call calculate_alpha_from_flux() to find α")
    print(f"  3. Use α to compute mixing length: l = α * H_p")
    print(f"  4. Apply convective adjustment based on F_c(α)")
else:
    print("\nNo valid solutions found in parameter space.")
    print("Try adjusting:")
    print("  - Increase F_need (increase F_tot or decrease F_rad)")
    print("  - Use more superadiabatic temperature profiles")
    print("  - Check physical parameters are appropriate")

print("="*60)


## Summary

This notebook explored the MLT mixing-length parameter α by solving the flux balance equation. Key findings:

- **Single layer tests** validated the calculation for specific cases
- **Flux sensitivity** showed how α scales with required convective flux  
- **Parameter space exploration** revealed how α varies with T/P profiles
- **Results exported** for integration into radiative transfer codes

### Next Steps:

1. Adjust parameters above to match your specific atmosphere
2. Run parameter space scan with your flux values
3. Use results to implement MLT in your RT code
4. Validate against observations or detailed models
